##Projet Final Estimation Prix Immobilier

Dans ce notebook, d'abord, nous récuppérons et traitons des données de transactions immobiliéres des années de 2015 à 2020

Ces données comportent des informations pour chaque transaction sur la valeur du bien, la surface du bien, Surface terrain, nombre de chambres, Code Département, adresse, Type de local.
Ces données seront utilisées ultérieurement lors de l'étape d'analyse des données.

Ensuite, On les fusionnes avec les données de communes globales (démoghraphie, revenus, géospatials) pour préparer la base de données d'entrainement pour le modéle machine learning de pérdiction.

En bas, une présentations sur les parties de ce code :
- Traiter et nettoyer les données fonciéres.
- EDA : : explorer les données
- Jointure des données fonciéres avec les données sur les communes (Partie prétraitement donnéescommunes
- EDA : explorer les données
- Construire les modéles machine learning 
- Comparaison entre les différents modéles
- Conclusion
- Pistes d'amélioration

In [ ]:
#importer les librairies nécessaires
import pandas as pd
import numpy as np
from datetime import datetime
import missingno as msno
import glob

import seaborn as sns
import matplotlib.pyplot as plt
import seaborn

In [ ]:
#installer le package xgboost
!pip install xgboost

  Using cached xgboost-1.3.3-py3-none-win_amd64.whl (95.2 MB)


# 1. Importation des données de transactions fonciéres

In [ ]:
# les columns a supprimer
col_include = ['date_mutation', 'nature_mutation', 'valeur_fonciere',
               'adresse_numero',  'code_postal', 'code_commune', 'nom_commune',
               'code_departement', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati',
               'nombre_pieces_principales','surface_terrain']
# transformer la columne en datetime
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
data_types = {'code_postal':'string',
              'adresse_numero': 'Int64',
              'nombre_lots':'Int16',
              'surface_reelle_bati': 'Float32',
              'nombre_pieces_principales': 'Int8',
              'surface_terrain':'Float32',
              'code_commune':'object'}
# dtype code postal string
valeur_fonciere_geo_2020 = pd.read_csv('datasets/full2020.csv', sep=",", 
                                   parse_dates=['date_mutation'], date_parser=dateparse, 
                                   dtype=data_types,usecols=col_include)
valeur_fonciere_geo_2019 = pd.read_csv('datasets/full2019.csv', sep=",", 
                                   parse_dates=['date_mutation'], date_parser=dateparse, 
                                   dtype=data_types,usecols=col_include)
valeur_fonciere_geo_2018 = pd.read_csv('datasets/full2018.csv', sep=",", 
                                   parse_dates=['date_mutation'], date_parser=dateparse, 
                                   dtype=data_types,usecols=col_include)
valeur_fonciere_geo_2017 = pd.read_csv('datasets/full2017.csv', sep=",", 
                                   parse_dates=['date_mutation'], date_parser=dateparse, 
                                   dtype=data_types,usecols=col_include)
valeur_fonciere_geo_2016 = pd.read_csv('datasets/full2016.csv', sep=",", 
                                   parse_dates=['date_mutation'], date_parser=dateparse, 
                                   dtype=data_types,usecols=col_include)
valeur_fonciere_geo_2015 = pd.read_csv('datasets/full2015.csv', sep=",", 
                                   parse_dates=['date_mutation'], date_parser=dateparse, 
                                   dtype=data_types,usecols=col_include)

C:\Users\Administrateur\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
#jointure de la 
frames = [valeur_fonciere_2020, valeur_fonciere_2019, valeur_fonciere_2018, valeur_fonciere_2017
         , valeur_fonciere_2016, valeur_fonciere_2015]
df_foncier = pd.concat(frames)

In [ ]:
frames_geo = [valeur_fonciere_geo_2020, valeur_fonciere_geo_2019, valeur_fonciere_geo_2018, 
              valeur_fonciere_geo_2017, valeur_fonciere_geo_2016, valeur_fonciere_geo_2015]
df_geo_foncier = pd.concat(frames_geo)

In [ ]:
df_geo_foncier.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16359982 entries, 0 to 2750304
Data columns (total 14 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   date_mutation              datetime64[ns]
 1   nature_mutation            object        
 2   valeur_fonciere            float64       
 3   adresse_numero             Int64         
 4   code_postal                string        
 5   code_commune               object        
 6   nom_commune                object        
 7   code_departement           object        
 8   nombre_lots                Int16         
 9   code_type_local            float64       
 10  type_local                 object        
 11  surface_reelle_bati        Float32       
 12  nombre_pieces_principales  Int8          
 13  surface_terrain            Float32       
dtypes: Float32(2), Int16(1), Int64(1), Int8(1), datetime64[ns](1), float64(2), object(5), string(1)
memory usage: 1.6+ GB


# 2. Préparation et netoyyage des données

In [ ]:
# Drop duplicates
initial_length = len(df_geo_foncier)
df_geo_foncier.drop_duplicates(inplace=True)
print("Suppression des doublons: {}".format(initial_length-len(df_geo_foncier)))

Suppression des doublons: 1639201


In [ ]:
print('origine',len(df_geo_foncier.index))
df_geo_foncier = df_geo_foncier.groupby(['date_mutation','valeur_fonciere']).filter(lambda x: len(x) == 1)
print('apres',len(df_geo_foncier.index))

origine 14720781
apres 1009960


Construire la dataframe de maison et appartement

In [ ]:
# df_MA = maison + appartment
df_geo_foncier.rename(columns={'code_commune':'Code INSEE'}, inplace=True)
df_MA = df_geo_foncier[(df_geo_foncier['type_local'].isin(['Maison','Appartement']))&(df_geo_foncier['nature_mutation']=='Vente')]

df_MA.head()

,date_mutation,nature_mutation,valeur_fonciere,adresse_numero,code_postal,Code INSEE,nom_commune,code_departement,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain
4,2020-01-22,Vente,209950.0,11,01000,01053,Bourg-en-Bresse,1,0,1.0,Maison,78.0,5,242.0
9,2020-01-16,Vente,207170.0,10,01000,01053,Bourg-en-Bresse,1,1,1.0,Maison,106.0,4,<NA>
20,2020-01-30,Vente,164650.0,37,01000,01053,Bourg-en-Bresse,1,0,1.0,Maison,90.0,4,212.0
44,2020-01-16,Vente,167490.0,5065,01340,01229,Malafretaz,1,0,1.0,Maison,160.0,5,1278.0
63,2020-01-13,Vente,153248.0,1671,01290,01334,Saint-André-d'Huiriat,1,0,1.0,Maison,203.0,6,1673.0


# Clean data

Présentation statistique les pourcentage des vides

In [3]:
# Missing assessment function 
def missing_values_assessment(df):
    nan_values = df.isnull().sum().sum()
    print('Nombre d\'observations: {:,}'.format(len(df.index)))
    print('Nombre de valeurs: {:,}'.format(df.size))
    print('Valeurs manquantes: {:,}'.format(nan_values))
    print('Qualité des données: {}%'.format(100-round((nan_values/df.size)*100,2)))
    print('Type de données:\n {}%'.format(df.dtypes.value_counts()))
    analysis = {'Manquant': df.isnull().sum(),
                'Manquant %':round((df.isnull().sum()/len(df))*100, 2),
                'Type':df.dtypes
               }
    return pd.DataFrame(analysis)

In [4]:
missing_values_assessment(df_MA).sort_values('Manquant %', ascending=False)

NameError: ignored

Supprimer les columns vides

In [ ]:
import missingno as msno
msno.matrix(df_MA, sort='ascending')

In [ ]:
df_MA['surface_terrain'].fillna(df_MA['surface_reelle_bati'], inplace=True)


In [ ]:
import missingno as msno
msno.matrix(df_MA, sort='ascending')

**Imputation**

In [ ]:
df_MA.dropna(how='any', axis=0, inplace=True)

Ajouter les columns Years et Months

In [ ]:
df_MA['Year'] = pd.DatetimeIndex(df_MA['date_mutation']).year
df_MA['Month'] = pd.DatetimeIndex(df_MA['date_mutation']).month

In [ ]:
df_MA['Prix_m2']=df_MA['valeur_fonciere']/df_MA['surface_reelle_bati']

In [ ]:
df_MA.info()

Export the data

In [ ]:
df_MA.to_csv(r'datasets/fonciere_geo.csv', index = None, header=True)

# EDA

In [1]:
df_MA['code_departement'].unique()

NameError: ignored

In [2]:
df_MA_IDF=df_MA[(df_MA['code_departement'].isin([75,91,92,93,94,95,77,78]))]
df_MA_IDF.head()

NameError: ignored

In [ ]:
#Quality correlation matrix
k = 8 #number of variables for heatmap
cols = df_MA_IDF.corr().nlargest(k, 'valeur_fonciere')['valeur_fonciere'].index
cm = df_MA_IDF[cols].corr()
plt.figure(figsize=(10,6))
sns.heatmap(cm, annot=True, cmap = 'viridis')

In [ ]:
sns.pairplot(data=df_MA_IDF)

Mutation par Mois

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(x=df_MA_IDF.Month)
ax.set_title('Nombre de mutation par mois')
ax.set_xlabel('Mois')
ax.set_ylabel('Nombre de mutation')

In [ ]:
plt.figure(figsize=(9,5))
df_MA_IDF['jour_de_semaine'] = df_MA_IDF['date_mutation'].map(lambda x: x.dayofweek)
ax = sns.countplot(x=df_MA_IDF['jour_de_semaine'])
ax.set_title('Nombre de mutation par jour de semaine')
ax.set_xlabel('Jour de semaine')
ax.set_xticklabels(['Lundi','Mardi','Mercredi','Jeudi','Vendredi','Samedi','Dimanche'],rotation=45)
ax.set_ylabel('Nombre de mutation')

Detect the outliers

In [ ]:
print('The "Surface terrain" where we reach the 90th/10th quantile is :',
      df_MA_IDF['surface_terrain'].quantile(q=0.95),df_MA_IDF['surface_terrain'].quantile(q=0.10),'m2',
      '\n',
      'The "Surface reelle bati" where we reach the 95th quantile is :',
      df_MA_IDF['surface_reelle_bati'].quantile(q=0.95),'m2',
      '\n',
     'The "Nombre pieces principales" where we reach the 99,95th quantile is :',
      df_MA_IDF['nombre_pieces_principales'].quantile(q=0.9995),'piece',
     '\n',
     'The "Valeur fonciere" where we reach the 99,95th quantile is :',
      df_MA_IDF['valeur_fonciere'].quantile(q=0.8),'euros')

In [ ]:
df_test = df_MA_IDF[(df_MA_IDF['surface_terrain']<600.0)&(df_MA_IDF['surface_terrain']>50.0)]
plt.boxplot(df_test['surface_terrain'])


In [ ]:
df_test = df_MA_IDF[(df_MA_IDF['surface_reelle_bati']<162)&(df_MA_IDF['surface_reelle_bati']>5)]
plt.boxplot(df_test['surface_reelle_bati'])

Outliers pour Nombre pieces principales

In [ ]:
df_test = df_MA_IDF[(df_MA_IDF['nombre_pieces_principales']<11)&(df_MA_IDF['nombre_pieces_principales']>1)]
plt.boxplot(df_test['nombre_pieces_principales'])

In [ ]:
df_test = df_MA_IDF[(df_MA_IDF['valeur_fonciere']<695000)&(df_MA_IDF['valeur_fonciere']>50000)]
plt.boxplot(df_test['valeur_fonciere'])

Construire le dataframe final

The "Surface terrain" where we reach the 90th/10th quantile is : 702.0 159.0 m2 
 The "Surface reelle bati" where we reach the 95th quantile is : 185.0 m2 
 The "Nombre pieces principales" where we reach the 99,95th quantile is : 13.0 piece 
 The "Valeur fonciere" where we reach the 99,95th quantile is : 655119.9999999998 euros

In [ ]:
df_fonciere_final = df_MA_IDF[(df_MA_IDF['valeur_fonciere']<695000)&(df_MA_IDF['valeur_fonciere']>50000)&
                     (df_MA_IDF['nombre_pieces_principales']<11)&(df_MA_IDF['nombre_pieces_principales']>1)&
                       (df_MA_IDF['surface_terrain']<600)&(df_MA_IDF['surface_terrain']>50)&
                     (df_MA_IDF['surface_reelle_bati']<162)&(df_MA_IDF['surface_reelle_bati']>5)]

df_fonciere_final.head()

In [ ]:
df_fonciere_final.drop(['date_mutation','nature_mutation','code_postal','type_local'],axis = 1, inplace = True)

In [ ]:
df_fonciere_final.head()

Concat data popu 

In [ ]:
all_data=pd.read_csv('datasets/all_dataset.csv')
all_data.head()

In [ ]:
#test_df = df_final[df_final['Code INSEE']=='94003']

df_merged = pd.merge(df_fonciere_final,all_data,on='Code INSEE',how='left', indicator=True)
df_merged.head()

In [ ]:
df_merged[df_merged['code_departement']==75]

In [ ]:
# Define subplots object
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10,10) , gridspec_kw={'hspace': 0.5})

fig.suptitle('Distribution of the Communes acrosss the studied features', fontsize=16)

# Plot density
sns.boxenplot(df_merged['density'],ax=ax1)
ax1.set_xlabel('Distribution of the density of population (in uninhabitant/km2). This plot was cropped')
ax1.set_ylabel('Commune \n frequency')
ax1.set_xlim(0,1000)

# Plot revenue_median
sns.boxenplot(df_merged['revenue_median'],ax=ax2)
ax2.set_xlabel('Distribution of the median standard of living of households (in €/an)')
ax2.set_ylabel('Commune \n frequency')

# Plot age_median
sns.boxenplot(df_merged['age_median'],ax=ax3)
ax3.set_xlabel('Distribution of the age median')
ax3.set_ylabel('Commune \n frequency')

# Plot Prix_m2
sns.boxenplot(df_merged['Prix_m2'],ax=ax4)
ax4.set_xlabel('Distribution of the mean price of houses per m2 ( in €/m2). This plot was cropped ')
ax4.set_ylabel('Commune \n frequency')
ax4.set_xlim(0,6000)

In [ ]:
print(df_merged.shape)

In [ ]:
print(df_merged.shape)
df_merged.dropna(how='any', axis=0, inplace=True)
print(df_merged.shape)

In [ ]:
df_merged.to_csv('datasets/df_merged_dataset.csv', index=False)

**Encode**

In [ ]:
df_merged.columns

In [ ]:
numeric_features = ['nombre_lots','surface_reelle_bati','nombre_pieces_principales','surface_terrain','superficie_km2',
                    'age_median','revenue_median','population','density']
categorical_features = ['adresse_numero','code_type_local','Month','code_departement']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression


np.random.seed(0)


numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])


categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


In [ ]:
col_X = numeric_features+categorical_features

X = df_merged[col_X]
y = df_merged['valeur_fonciere']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

LinearRegression

In [ ]:
model1 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('lregressor', LinearRegression())])

model1.fit(X_train, y_train)
print("model R2 score: %.3f" % model1.score(X_test, y_test))

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model1.predict(X_test)
MSE = mean_squared_error(y_test, y_pred)
print('model1 MSE is: ', MSE)

In [ ]:
print(21090385107.22951-15907330336.873158)
print(21090385107.22951-19456898431.83947)
print(21090385107.22951-9878000087.029533)


xgboost

In [ ]:
import xgboost as xg 

model2 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('xgbregressor', xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 200, seed = 0))])

model2.fit(X_train, y_train)
print("model score: %.3f" % model2.score(X_test, y_test))

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred2 = model2.predict(X_test)
MSE2 = mean_squared_error(y_test, y_pred2)
print('model2 MSE is: ', MSE2)

RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model3 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rfregressor', RandomForestRegressor(max_depth=10,n_estimators = 200))])

model3.fit(X_train, y_train)
print("model score: %.3f" % model3.score(X_test, y_test))

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred3 = model3.predict(X_test)
MSE3 = mean_squared_error(y_test, y_pred3)
print('model3 MSE is: ', MSE3)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model4 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rfregressor', RandomForestRegressor(max_depth=10,n_estimators = 200, verbose = 1))])

model4.fit(X_train, y_train)
print("model score: %.3f" % model4.score(X_test, y_test))

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred4 = model4.predict(X_test)
MSE4 = mean_squared_error(y_test, y_pred4)
print('model MSE is: ', MSE4)